In [3]:
# print cwd
import os
print(os.getcwd())

c:\Users\cayde\Desktop\CF_2_more_orginized\Projects\Geo\scripts


In [4]:
#orginal file name : exploringimmigrationCrimeData_nopii.ipynb
# path to said file : C:\Users\cayde\Desktop\CF\exploringimmigrationCrimeData_nopii.ipynb

#import libraries
import pandas as pd
import numpy as np

ApprissCrime_df = pd.read_csv(r'../data/batch7/Copy of Return FIle, February.csv')

#here we read in our data
NCIC_dic_df = pd.read_csv(r"../data\NCIC_code_list - Copy.csv")

# read in the dic file
dict_df = pd.read_csv(r"../data\diction_NCICcodes_descriptions_CrimeTiers_CDCTiers_as_csv - Copy.csv")

#add columns to the dataframe
ApprissCrime_df['max_tier_charge'] = np.nan
ApprissCrime_df['max_tier_charge_desc'] = np.nan
ApprissCrime_df['immigration_charge_present'] = np.nan


In [5]:
# if any row of the column 'BookingChargeNCICOffenseCode' contains a '[' remove it
ApprissCrime_df['BookingChargeNCICOffenseCode'] = ApprissCrime_df['BookingChargeNCICOffenseCode'].str.replace('[', '')

# if any row of the column 'BookingChargeNCICOffenseCode' contains a ']' remove it
ApprissCrime_df['BookingChargeNCICOffenseCode'] = ApprissCrime_df['BookingChargeNCICOffenseCode'].str.replace(']', '')

# here we are creating a subset of the raw data that has the CorrelationID, BookingDate, and unrolled NCICcodes per row
# each rows unrolled NCIC offense codes
unrolled_NCICCodes_df = ApprissCrime_df['BookingChargeNCICOffenseCode'].str.split(',', expand=True)

# ClientBillingID
# SenderId
#pull the correlationId and bookingdate columns for the data set and join them to the unrolled NCIC codes
correlationID_column = ApprissCrime_df['SenderId']
BookingDate_column = ApprissCrime_df['BookingDate']

unrolled_NCICCodes_df.insert(0, 'BookingDate', BookingDate_column)
unrolled_NCICCodes_df.insert(0, 'SenderId', correlationID_column)

# if any value in the dataframe 'unrolled_NCICCodes_df' is ' ' replace it with ''
unrolled_NCICCodes_df = unrolled_NCICCodes_df.replace(' ', '', regex=True)
unrolled_NCICCodes_df = unrolled_NCICCodes_df.fillna('')
# if any value in the dataframe 'unrolled_NCICCodes_df' is 'VANDALISM' replace it with ''
unrolled_NCICCodes_df = unrolled_NCICCodes_df.replace('VANDALISM', '', regex=True)
unrolled_NCICCodes_df = unrolled_NCICCodes_df.replace('ASSAULTANDBATTERY', '', regex=True)

# if any value in the dataframe 'unrolled_NCICCodes_df' is '' replace it with nan
#unrolled_NCICCodes_df = unrolled_NCICCodes_df.replace('' , np.nan)

# here we are gonna make a distrobustion of counts of NCIC codes
column_names_1 = ['NCICcode_count']
index_names = []
NCICcode_counts_df = pd.DataFrame(columns = column_names_1,)

for index, row in NCIC_dic_df.iterrows():
    if(row['NCIC Offense Code'] is not None and row['NCIC Offense Code'] != ''):
        df = pd.DataFrame({"NCICcode_count":[0]})
        NCICcode_counts_df = pd.concat([NCICcode_counts_df, df])
        index_names.append(row['NCIC Offense Code'])

NCICcode_counts_df = pd.DataFrame(NCICcode_counts_df).set_index([index_names])

#get the number of columns in the dataframe 'unrolled_NCICCodes_df'
num_of_columns = len(unrolled_NCICCodes_df.columns) - 3 # -3 because we dont want to count the index, senderId, and BookingDate columns


for ind, row in unrolled_NCICCodes_df.iterrows():
    #print(row['CorrelationID'], row['BookingDate'], row[0])
    for i in range(0, num_of_columns):
        if(row[i] is not None and row[i] != ''):
            NCICcode = int(row[i])
            NCICcode_counts_df.at[NCICcode, 'NCICcode_count'] = NCICcode_counts_df.at[NCICcode, 'NCICcode_count'] + 1

no_examples = []
counter = 0
for ind, row in NCICcode_counts_df.iterrows():
    if(row['NCICcode_count'] == 0):
        no_examples.append(row.name)
    else:
        counter = counter + 1

print("455 ncic codes")
print('NCIC codes that never show up in dataset: ' + str(len(no_examples)))
print('NCIC codes that show up in dataset: ' + str(counter))

C:\Users\cayde\AppData\Local\Temp\ipykernel_3232\1141814517.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ApprissCrime_df['BookingChargeNCICOffenseCode'] = ApprissCrime_df['BookingChargeNCICOffenseCode'].str.replace('[', '')
C:\Users\cayde\AppData\Local\Temp\ipykernel_3232\1141814517.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ApprissCrime_df['BookingChargeNCICOffenseCode'] = ApprissCrime_df['BookingChargeNCICOffenseCode'].str.replace(']', '')


455 ncic codes
NCIC codes that never show up in dataset: 413
NCIC codes that show up in dataset: 42


In [6]:
# language code python
#Here we are going to refine our dictionary to match the follwoing schema 
# NCICCODE(index) | Crime Tier | CDC Crime Category 

# dict_df = dict_df.drop(["Description of Crime","Notes:"], axis=1)
dict_df = dict_df.set_index('NCIC Offense Code')
dict_df

#here we are adding tier 5 so we can track immigration crimes
dict_df.loc[dict_df['Notes:'] == 'Immigration crime', 'Tier'] = "5"

# #here we are going to change the following tiers of the following booking descriptions
dict_df.loc[dict_df['Description of Crime'] == 'CRUELTY TOWARD CHILD', 'Tier'] = 3
dict_df.loc[dict_df['Description of Crime'] == 'CONTRIB DELINQ MINOR', 'Tier'] = 3
dict_df.loc[dict_df['Description of Crime'] == 'OBSCENE MATERIAL – POSSESS', 'Tier'] = 3
dict_df.loc[dict_df['Description of Crime'] == 'PROSTITUTION', 'Tier'] = 3
dict_df.loc[dict_df['Description of Crime'] == 'FAMILY OFFENSE', 'Tier'] = 4
dict_df.loc[dict_df['Description of Crime'] == 'GAMBLING', 'Tier'] = 4
dict_df.loc[dict_df['Description of Crime'] == 'Indecent Exposure (to minors and adults)', 'Tier'] = 2




In [7]:
# add columns to the dataframe

#here we are going to add max tier to each booking charge
# we are also going to add max tier description 
# we are also going to add immigration charge present

def check_tier_against_last(old_val, new_val):
    #print(old_val, new_val)
    if(old_val == 'NaN' or old_val is None):
        return new_val
    if(new_val == 'NaN' or new_val is None):
        return 'NaN'
    # if the 'old_val' is bigger than or equal to the 'new_val' return the 'new_val'
    if(old_val >= new_val):
        return new_val
    else:
        return old_val

def check(val):
    if(val == 1.0):
        return 1
    if(val == 2.0):
        return 2
    if(val == 3.0):
        return 3
    if(val == 3):
        return 3
    if(val == 4.0):
        return 4
    if(val == 4):
        return 4
    if(val == '5'):
        return 5
    if(val == None):
        return None
    if(val == 'NaN'):
        return None

#itterate through the dataframe
for index, row in unrolled_NCICCodes_df.iterrows():
    for i in range(0, num_of_columns):
        if(row[i] is not None and row[i] != ''):
            NCICcode = int(row[i])
            TierVal = dict_df.at[NCICcode, "Tier"]
            description = dict_df.at[NCICcode, "Description of Crime"]
            TierVal = check(TierVal)
            if(TierVal == 5): # this handles immigration charge present
                ApprissCrime_df.at[index, 'immigration_charge_present'] = 1
            if(pd.isnull(ApprissCrime_df.at[index, 'max_tier_charge'])):
                ApprissCrime_df.at[index, 'max_tier_charge'] = TierVal
                ApprissCrime_df.at[index, 'max_tier_charge_desc'] = description
            else:
                if(pd.isnull(ApprissCrime_df.at[index, 'max_tier_charge'])) == True:
                    print(ApprissCrime_df.at[index, 'max_tier_charge'], TierVal)
                    if(TierVal <= ApprissCrime_df.at[index, 'max_tier_charge']):
                        ApprissCrime_df.at[index, 'max_tier_charge'] = TierVal
                        ApprissCrime_df.at[index, 'max_tier_charge_desc'] = description

ApprissCrime_df.loc[ApprissCrime_df['BookingChargeDescription'] == 'IMMIGRATION', 'immigration_charge_present'] = 1
ApprissCrime_df.loc[ApprissCrime_df['BookingChargeDescription'] == 'ICE DETENTION', 'immigration_charge_present'] = 1
ApprissCrime_df.loc[ApprissCrime_df['BookingChargeDescription'] == 'IMMIGRATION & CUSTOMS DETAINEE', 'immigration_charge_present'] = 1
ApprissCrime_df.loc[ApprissCrime_df['BookingChargeDescription'] == 'WARRANT FOR DEPORTATION', 'immigration_charge_present'] = 1
ApprissCrime_df.loc[ApprissCrime_df['BookingChargeDescription'] == 'INTRANSIT/ILLEGAL RE-ENTRY', 'immigration_charge_present'] = 1
ApprissCrime_df.loc[ApprissCrime_df['BookingChargeDescription'] == 'IMMIGRATION VIOLATIONS-90Z', 'immigration_charge_present'] = 1
ApprissCrime_df.loc[ApprissCrime_df['BookingChargeDescription'] == 'IMMIGRANT W/O VISA', 'immigration_charge_present'] = 1
ApprissCrime_df.loc[ApprissCrime_df['BookingChargeDescription'] == 'ICE HOLD', 'immigration_charge_present'] = 1
ApprissCrime_df.loc[ApprissCrime_df['BookingChargeDescription'] == 'BORDER PATROL HOLD', 'immigration_charge_present'] = 1

In [9]:
# write the dataframe to csv
ApprissCrime_df.to_csv(r"../data/batch7/out/file_February_withappends.csv")